# AstraForge Toolkit — Local API smoke test

Quick sanity checks for the published package against a locally running AstraForge API, plus the same quick-start snippets from the README.

Prereqs:
- Backend running at `http://localhost:8000` (e.g., `make backend-serve`)
- An API key (create via the in-app API Keys screen or `/api/api-keys/`)
- Package installed (`pip install -e ../astraforge-python-package` or from PyPI)
- For the DeepAgent example, set `OPENAI_API_KEY` (or configure your preferred model) so LangChain can call a chat model.


In [ ]:
# Install the local package in editable mode (run from the examples/ folder)
%pip install -e ..


> If you see `ImportError: cannot import name '_DirectlyInjectedToolArg'`, your environment's LangChain/LangGraph stack is too old. Run the next cell to bump to the DeepAgent-compatible versions. A clean virtualenv is recommended if other packages pin older LangChain releases.


In [7]:
# Align LangChain/LangGraph versions for DeepAgent
%pip install "langchain>=1.1.0" "langchain-core>=1.1.0" "langgraph>=1.0.4" "langgraph-prebuilt>=1.0.5"


Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_core-1.1.0-py3-none-any.whl.metadata (3.6 kB)
Using cached langchain_core-1.1.0-py3-none-any.whl (473 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.80
    Uninstalling langchain-core-0.3.80:
      Successfully uninstalled langchain-core-0.3.80
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.3.24 requires langchain-core<1.0.0,>=0.3.65, but you have langchain-core 1.1.0 which is incompatible.
langchain-text-splitters 0.3.5 requires langchain-core<0.4.0,>=0.3.29, but you have langchain-core 1.1.0 which is incompatible.
langchain-ollama 0.3.8 requires langchain-core<1.0.0,>=0.3.76, but you have langchain-core 1.1.0 which is incompatible.
langchain-community 0.3.7 requires langchain<0.4.0,>=0.3.7, 

In [ ]:
%pip install python-dotenv


In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
import os

BASE_URL = "http://localhost:8000/api"
API_KEY = os.getenv("ASTRA_FORGE_API_KEY")

### Create a sandbox-backed DeepAgent (README quick start)

Same as the README example: configure a SandboxBackend, pick a chat model, and build a DeepAgent that can use sandbox tools.


In [3]:
# Install a model provider helper for the example (skip if already installed)
%pip install langchain-openai

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.1.0
    Uninstalling langchain-core-1.1.0:
      Successfully uninstalled langchain-core-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepagents 0.2.8 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.
langchain-community 0.3.7 requires langchain<0.4.0,>=0.3.7, but you have langchain 1.1.0 which is incompatible.
langchain-community 0.3.7 requires langsmith<0.2.0,>=0.1.125, but you have langsmith 0.4.29 which is incompatible.
langchain 1.1.0 requires langchain-core<2.0.0,>=1.1.0, but you have langchain-core 0.3.80 which is incompat

In [6]:
from deepagents import create_deep_agent
from langchain_openai import ChatOpenAI
from astraforge_toolkit import (
    SandboxBackend,
    sandbox_shell,
    sandbox_python_repl,
    sandbox_open_url_with_playwright,
    sandbox_view_image,
)

def backend_factory(rt):
    return SandboxBackend(
        rt,
        base_url=BASE_URL,
        api_key=API_KEY,
        # optional: session_params={"image": "astraforge/codex-cli:latest"},
    )

model = ChatOpenAI(model="gpt-4o-mini")  # or any chat model you prefer
tools = [
    sandbox_shell,
    sandbox_python_repl,
    sandbox_open_url_with_playwright,
    sandbox_view_image,
]

deep_agent = create_deep_agent(
    model=model,
    backend=backend_factory,
    tools=tools,
)

deep_agent.invoke(
    {"messages": [{"role": "user", "content": "List files in /workspace"}]},
)


/home/jeremy/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name '_DirectlyInjectedToolArg' from 'langchain_core.tools.base' (/home/jeremy/.local/lib/python3.11/site-packages/langchain_core/tools/base.py)

### Call DeepAgent over HTTP (README quick start)

The client hits the REST API directly; it matches the README snippet and streams chunks.


In [ ]:
from astraforge_toolkit import DeepAgentClient

client = DeepAgentClient(base_url=BASE_URL, api_key=API_KEY)
conv = client.create_conversation()

for chunk in client.stream_message(conv.conversation_id, "Hello, sandbox!"):
    print(chunk)


### Optional: send a single non-streaming message

Useful if you just want the final response payload.


In [ ]:
response = client.send_message(
    conversation_id=conv.conversation_id,
    messages=[{"role": "user", "content": "Hello from notebook"}],
    stream=False,
)
response
